# Here is the refactored solution. I have separated the concerns into three distinct layers:
1.  **The Data Contract:** explicit `dataclasses` defining exactly what goes in and comes out.
2.  **The Engine:** A purely mathematical class (`AlphaEngine`) containing the logic, with no widget/plotting dependencies.
3.  **The UI:** A cleaned-up dashboard function that simply sends inputs to the Engine and visualizes the Output

### Following is the reverse chronological fix log (most recent entry is at the top )

```
To see the full dataframe of all tickers (both those that passed and those that failed) for a specific date, we need to capture a snapshot of the universe inside the `_get_eligible_universe` method.

I have updated the **`AlphaEngine`** class below.
```

```
To verify that the relative percentile logic is working, we can modify the `AlphaEngine` to report exactly **how the cutoff was calculated** for the specific start date.

We want to see evidence that:
1.  In earlier years (e.g., 2005), the volume cutoff is lower (e.g., $200k).
2.  In later years (e.g., 2024), the volume cutoff is higher (e.g., $5M).

Here is the updated `AlphaEngine` and `UI` code. I have added a **"Audit Log"** feature. When you run the tool, it will now print exactly what the Dollar Volume Threshold was for that specific day.
```

```
The best way to solve this is to switch from a **Fixed Dollar Threshold** (e.g., "$1 Million") to a **Relative Percentile Threshold** (e.g., "Top 50% of the market").

In 2004, a stock trading $200k might have been in the top 50% of liquid stocks. In 2024, that same $200k is illiquid garbage. Using a percentile automatically adjusts for inflation and market growth over time.

Here is how to modify your code to support this.
```

```
To fix this, we need to pass the **actual** calculated start date (the trading day the engine "snapped" to) back from the `AlphaEngine` to the UI. Then, the UI can compare the *Requested Date* vs. the *Actual Date* and display the warning message if they differ.

Here is the plan:
1.  **Update `EngineOutput`**: Add a `start_date` field to the dataclass.
2.  **Update `AlphaEngine.run`**: Populate this new field with `safe_start_date`.
3.  **Update `plot_walk_forward_analyzer`**: Add logic to compare the user's input date with the engine's returned date and print the "Info" message if they are different.

Here is the updated code (Sections C, D, and E have changed):
```

```
I have updated the `AlphaEngine.run` method. specifically inside the `if inputs.mode == 'Manual List':` block. It now iterates through every manual ticker and performs two checks:
1.  **Existence**: Is the ticker in the database?
2.  **Availability**: Does the ticker have a valid price on the specific `Start Date`?

If any ticker fails, it compiles a specific error message explaining why (e.g., "No price data on start date") and aborts the calculation immediately.  
```

```
The `snapshot_df` contains **every single feature** calculated by your `generate_features` function for that specific day, plus the new audit columns we added.

Here is exactly what is inside that DataFrame:

### 1. The Core Features (from `generate_features`)
*   **`TR`**: True Range
*   **`ATR`**: Average True Range
*   **`ATRP`**: Average True Range Percent (Volatility)
*   **`RollingStalePct`**: How often the price didn't move or volume was 0.
*   **`RollMedDollarVol`**: Median Daily Dollar Volume (Liquidity).
*   **`RollingSameVolCount`**: Data quality check for repeated volume numbers.

### 2. The Audit Columns (Added during filtering)
*   **`Calculated_Cutoff`**: The specific dollar amount required to pass on that day.
*   **`Passed_Vol_Check`**: `True` if the ticker met the liquidity requirement.
*   **`Passed_Final`**: `True` if it passed **all** checks (Liquidity + Stale + Quality).

=========================================

Here are the formulas translated directly into the Python `pandas` code used in your `generate_features` function.

I have simplified the code slightly to assume a single ticker context (removing the `groupby` wrapper) so you can see the raw math clearly.

### 1. True Range (TR)
Calculates the maximum of the three price differences.

prev_close = df_ohlcv['Adj Close'].shift(1)

# The three components
diff1 = df_ohlcv['Adj High'] - df_ohlcv['Adj Low']
diff2 = (df_ohlcv['Adj High'] - prev_close).abs()
diff3 = (df_ohlcv['Adj Low'] - prev_close).abs()

# Taking the max of the three
tr = pd.concat([diff1, diff2, diff3], axis=1).max(axis=1)

### 2. Average True Range (ATR)
Uses an Exponential Weighted Mean (EWM) with a specific alpha smoothing factor.

# N = atr_period (e.g., 14)
# alpha = 1 / N
atr = tr.ewm(alpha=1/14, adjust=False).mean()

### 3. ATR Percent (ATRP)
Simple division to normalize volatility.

atrp = atr / df_ohlcv['Adj Close']

### 4. Rolling Stale Percentage
Checks if volume is 0 OR if High equals Low (price didn't move), then averages that 1 or 0 signal over the window.

# 1. Define the Stale Signal (1 for stale, 0 for active)
is_stale = np.where(
    (df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 
    1,  
    0
)

# 2. Calculate average over window (W=252)
rolling_stale_pct = pd.Series(is_stale).rolling(window=252).mean()

### 5. Rolling Median Dollar Volume
Calculates raw dollar volume, then finds the median over the window.

# 1. Calculate Daily Dollar Volume
dollar_volume = df_ohlcv['Adj Close'] * df_ohlcv['Volume']

# 2. Get Median over window (W=252)
roll_med_dollar_vol = dollar_volume.rolling(window=252).median()

### 6. Rolling Same Volume Count
Checks if today's volume is exactly the same as yesterday's (a sign of bad data), then sums those occurrences.

# 1. Check if Volume(t) - Volume(t-1) equals 0
# .diff() calculates current row minus previous row
has_same_volume = (df_ohlcv['Volume'].diff() == 0).astype(int)

# 2. Sum the errors over window (W=252)
rolling_same_vol_count = has_same_volume.rolling(window=252).sum()

```

================================  

In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import pprint

from dataclasses import dataclass, field
from typing import List, Dict, Optional, Any, Union
from collections import Counter
from datetime import datetime, date

# ==============================================================================
# SECTION A: CORE HELPER FUNCTIONS & FEATURE GENERATION
# (Unchanged from previous version)
# ==============================================================================
# ... (Keep generate_features, calculate_gain, calculate_sharpe, 
#      calculate_sharpe_atr, calculate_buy_and_hold_performance as is) ...

def generate_features(df_ohlcv: pd.DataFrame, atr_period: int = 14, quality_window: int = 252, quality_min_periods: int = 126) -> pd.DataFrame:
    # (Same as before)
    if not df_ohlcv.index.is_monotonic_increasing: df_ohlcv = df_ohlcv.sort_index()
    grouped = df_ohlcv.groupby(level='Ticker')
    prev_close = grouped['Adj Close'].shift(1)
    tr = pd.concat([df_ohlcv['Adj High'] - df_ohlcv['Adj Low'], abs(df_ohlcv['Adj High'] - prev_close), abs(df_ohlcv['Adj Low'] - prev_close)], axis=1).max(axis=1, skipna=False)
    atr = tr.groupby(level='Ticker').transform(lambda x: x.ewm(alpha=1/atr_period, adjust=False).mean())
    atrp = (atr / df_ohlcv['Adj Close']).replace([np.inf, -np.inf], np.nan)
    indicator_df = pd.DataFrame({'TR': tr, 'ATR': atr, 'ATRP': atrp})
    quality_temp_df = pd.DataFrame({'IsStale': np.where((df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 1, 0), 'DollarVolume': df_ohlcv['Adj Close'] * df_ohlcv['Volume'], 'HasSameVolume': (grouped['Volume'].diff() == 0).astype(int)}, index=df_ohlcv.index)
    rolling_result = quality_temp_df.groupby(level='Ticker').rolling(window=quality_window, min_periods=quality_min_periods).agg({'IsStale': 'mean', 'DollarVolume': 'median', 'HasSameVolume': 'sum'}).rename(columns={'IsStale': 'RollingStalePct', 'DollarVolume': 'RollMedDollarVol', 'HasSameVolume': 'RollingSameVolCount'}).reset_index(level=0, drop=True)
    return pd.concat([indicator_df, rolling_result], axis=1)

def calculate_gain(price_series): 
    if price_series.dropna().shape[0] < 2: return np.nan
    return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

def calculate_sharpe(return_series):
    if return_series.dropna().shape[0] < 2: return np.nan
    std = return_series.std()
    return (return_series.mean() / std * np.sqrt(252)) if std > 0 else 0.0

def calculate_sharpe_atr(return_series, atrp_series):
    if return_series.dropna().shape[0] < 2 or atrp_series.dropna().empty: return np.nan
    mean_atrp = atrp_series.mean()
    return (return_series.mean() / mean_atrp) if mean_atrp > 0 else 0.0

def calculate_buy_and_hold_performance(df_close, features_df, tickers, start_date, end_date):
    if not tickers: return pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)
    ticker_counts = Counter(tickers)
    initial_weights = pd.Series({t: c / len(tickers) for t, c in ticker_counts.items()})
    prices_raw = df_close[initial_weights.index.tolist()].loc[start_date:end_date]
    if prices_raw.dropna(how='all').empty: return pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)
    prices_norm = prices_raw.div(prices_raw.bfill().iloc[0])
    weighted_growth = prices_norm.mul(initial_weights, axis='columns')
    value_series = weighted_growth.sum(axis=1)
    return_series = value_series.pct_change()
    full_idx = pd.MultiIndex.from_product([initial_weights.index.tolist(), return_series.index], names=['Ticker', 'Date'])
    feat_subset = features_df.reindex(full_idx)['ATRP'].unstack(level='Ticker')
    atrp_series = (weighted_growth.div(value_series, axis='index').align(feat_subset, join='inner', axis=1)[0] * weighted_growth.div(value_series, axis='index').align(feat_subset, join='inner', axis=1)[1]).sum(axis=1)
    return value_series, return_series, atrp_series

# ==============================================================================
# SECTION B: METRIC REGISTRY
# ==============================================================================

def metric_price(d): return calculate_gain(d['calc_close'])
def metric_sharpe(d): 
    r = d['daily_returns']
    return (r.mean() / r.std() * np.sqrt(252)).replace([np.inf, -np.inf], np.nan).fillna(0)
def metric_sharpe_atr(d):
    return (d['daily_returns'].mean() / d['atrp']).replace([np.inf, -np.inf], np.nan).fillna(0)

METRIC_REGISTRY = {
    'Price': metric_price,
    'Sharpe': metric_sharpe,
    'Sharpe (ATR)': metric_sharpe_atr,
}

# ==============================================================================
# SECTION C: DATA CONTRACTS (The API)
# Updated EngineOutput to include actual start_date
# ==============================================================================

@dataclass
class EngineInput:
    mode: str
    start_date: pd.Timestamp
    calc_period: int
    fwd_period: int
    metric: str
    benchmark_ticker: str
    rank_start: int = 1
    rank_end: int = 10
    quality_thresholds: Dict[str, float] = field(default_factory=lambda: {'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10})
    manual_tickers: List[str] = field(default_factory=list)
    debug: bool = False

@dataclass
class EngineOutput:
    portfolio_series: pd.Series
    benchmark_series: pd.Series
    normalized_plot_data: pd.DataFrame
    tickers: List[str]
    initial_weights: pd.Series
    perf_metrics: Dict[str, float]
    results_df: pd.DataFrame
    start_date: pd.Timestamp # <--- NEW FIELD: The actual trading start date used
    calc_end_date: pd.Timestamp
    viz_end_date: pd.Timestamp
    error_msg: Optional[str] = None
    debug_data: Optional[Dict[str, Any]] = None

# ==============================================================================
# SECTION D: THE ALPHA ENGINE (The "Brain")
# This version saves a sorted dataframe called `universe_snapshot` into the debug data. It adds columns showing exactly which tickers passed or failed the specific thresholds.
# ==============================================================================

class AlphaEngine:
    def __init__(self, df_ohlcv: pd.DataFrame, master_ticker: str = 'SPY'):
        print("--- ⚙️ Initializing AlphaEngine ---")
        self.features_df = generate_features(df_ohlcv)
        print("Optimizing data structures...")
        self.df_close = df_ohlcv['Adj Close'].unstack(level=0)
        
        if master_ticker not in self.df_close.columns:
            master_ticker = self.df_close.columns[0]
            print(f"Warning: Master ticker not found. Using {master_ticker}")
            
        self.trading_calendar = self.df_close[master_ticker].dropna().index.unique().sort_values()
        print("✅ AlphaEngine Ready.")

    def run(self, inputs: EngineInput) -> EngineOutput:
        # --- A. Validate Dates ---
        try:
            start_idx = self.trading_calendar.searchsorted(inputs.start_date)
            if start_idx < 0: start_idx = 0
        except Exception:
            return self._error_result("Invalid Start Date")

        desired_end_idx = start_idx + inputs.calc_period + inputs.fwd_period
        if desired_end_idx >= len(self.trading_calendar):
            return self._error_result(f"Date range exceeds history.")

        safe_start_date = self.trading_calendar[start_idx]
        safe_calc_end_date = self.trading_calendar[start_idx + inputs.calc_period]
        safe_viz_end_date = self.trading_calendar[start_idx + inputs.calc_period + inputs.fwd_period]

        # --- B. Select Tickers ---
        tickers_to_trade = []
        results_table = pd.DataFrame()
        debug_dict = {}
        audit_info = {} 

        if inputs.mode == 'Manual List':
            validation_errors = []
            valid_tickers = []
            for t in inputs.manual_tickers:
                if t not in self.df_close.columns:
                    validation_errors.append(f"❌ {t}: Ticker not found.")
                    continue
                if pd.isna(self.df_close.at[safe_start_date, t]):
                    validation_errors.append(f"⚠️ {t}: No price data on start date.")
                    continue
                valid_tickers.append(t)
            
            if validation_errors: return self._error_result("\n".join(validation_errors))
            if not valid_tickers: return self._error_result("No valid tickers.")
            tickers_to_trade = valid_tickers
            results_table = pd.DataFrame(index=valid_tickers)
            
        else: # Ranking Mode
            eligible_tickers = self._get_eligible_universe(safe_start_date, inputs.quality_thresholds, audit_info)
            debug_dict['audit_liquidity'] = audit_info 
            
            if not eligible_tickers: return self._error_result("No tickers passed quality filters.")
            
            calc_close = self.df_close.loc[safe_start_date:safe_calc_end_date, eligible_tickers]
            idx_product = pd.MultiIndex.from_product([eligible_tickers, calc_close.index], names=['Ticker', 'Date'])
            feat_slice = self.features_df.reindex(idx_product).dropna(how='all')
            atrp_mean = feat_slice.groupby(level='Ticker')['ATRP'].mean()
            
            ingredients = { 'calc_close': calc_close, 'daily_returns': calc_close.pct_change(), 'atrp': atrp_mean }
            if inputs.metric not in METRIC_REGISTRY: return self._error_result(f"Metric '{inputs.metric}' not found.")
            metric_vals = METRIC_REGISTRY[inputs.metric](ingredients)
            sorted_tickers = metric_vals.sort_values(ascending=False)
            
            start_r = max(0, inputs.rank_start - 1)
            end_r = inputs.rank_end
            tickers_to_trade = sorted_tickers.iloc[start_r:end_r].index.tolist()
            if not tickers_to_trade: return self._error_result("No tickers generated from ranking.")

            results_table = pd.DataFrame({
                'Rank': range(inputs.rank_start, inputs.rank_start + len(tickers_to_trade)),
                'Ticker': tickers_to_trade,
                'Metric Value': sorted_tickers.loc[tickers_to_trade].values
            }).set_index('Ticker')

        # --- C. Performance Calculations ---
        p_val, p_ret, p_atrp = calculate_buy_and_hold_performance(self.df_close, self.features_df, tickers_to_trade, safe_start_date, safe_viz_end_date)
        b_val, b_ret, b_atrp = calculate_buy_and_hold_performance(self.df_close, self.features_df, [inputs.benchmark_ticker], safe_start_date, safe_viz_end_date)

        # --- D. Final Metrics ---
        plot_data = self.df_close[list(set(tickers_to_trade))].loc[safe_start_date:safe_viz_end_date]
        if not plot_data.empty: plot_data = plot_data / plot_data.bfill().iloc[0]
        calc_end_ts = safe_calc_end_date
        metrics = {}
        get_gain = lambda s: (s.iloc[-1] / s.iloc[0]) - 1 if len(s) > 0 else 0

        metrics['full_p_gain'] = get_gain(p_val)
        metrics['calc_p_gain'] = get_gain(p_val.loc[:calc_end_ts])
        metrics['fwd_p_gain'] = get_gain(p_val.loc[calc_end_ts:])
        metrics['full_p_sharpe_atr'] = calculate_sharpe_atr(p_ret, p_atrp)
        metrics['calc_p_sharpe_atr'] = calculate_sharpe_atr(p_ret.loc[:calc_end_ts], p_atrp.loc[p_ret.loc[:calc_end_ts].index])
        metrics['fwd_p_sharpe_atr'] = calculate_sharpe_atr(p_ret.loc[calc_end_ts:].iloc[1:], p_atrp.loc[p_ret.loc[calc_end_ts:].iloc[1:].index])
        
        if not b_ret.empty:
            metrics['full_b_gain'] = get_gain(b_val)
            metrics['calc_b_gain'] = get_gain(b_val.loc[:calc_end_ts])
            metrics['fwd_b_gain'] = get_gain(b_val.loc[calc_end_ts:])
            metrics['full_b_sharpe_atr'] = calculate_sharpe_atr(b_ret, b_atrp)
            metrics['calc_b_sharpe_atr'] = calculate_sharpe_atr(b_ret.loc[:calc_end_ts], b_atrp.loc[b_ret.loc[:calc_end_ts].index])
            metrics['fwd_b_sharpe_atr'] = calculate_sharpe_atr(b_ret.loc[calc_end_ts:].iloc[1:], b_atrp.loc[b_ret.loc[calc_end_ts:].iloc[1:].index])

        if not plot_data.empty: results_table['Fwd Gain'] = (plot_data.iloc[-1] / plot_data.loc[calc_end_ts]) - 1
        ticker_counts = Counter(tickers_to_trade)
        weights = pd.Series({t: c/len(tickers_to_trade) for t, c in ticker_counts.items()})

        if inputs.debug:
            trace_df = plot_data.copy()
            trace_df.columns = [f'Norm_Price_{c}' for c in trace_df.columns]
            trace_df['Norm_Price_Portfolio'] = p_val
            if not b_val.empty: trace_df[f'Norm_Price_Benchmark_{inputs.benchmark_ticker}'] = b_val
            debug_dict['portfolio_trace'] = trace_df

        return EngineOutput(
            portfolio_series=p_val, benchmark_series=b_val, normalized_plot_data=plot_data,
            tickers=tickers_to_trade, initial_weights=weights, perf_metrics=metrics,
            results_df=results_table, start_date=safe_start_date,
            calc_end_date=safe_calc_end_date, viz_end_date=safe_viz_end_date, debug_data=debug_dict
        )

    # --- UPDATED: CAPTURE SNAPSHOT ---
    def _get_eligible_universe(self, date_ts, thresholds, audit_container=None):
        avail_dates = self.features_df.index.get_level_values('Date').unique().sort_values()
        valid_dates = avail_dates[avail_dates <= date_ts]
        if valid_dates.empty: return []
        day_features = self.features_df.xs(valid_dates[-1], level='Date')

        # 1. Determine Dynamic Cutoff
        vol_cutoff = thresholds.get('min_median_dollar_volume', 0)
        percentile_used = "N/A"
        dynamic_val = 0
        
        if 'min_liquidity_percentile' in thresholds:
            percentile_used = thresholds['min_liquidity_percentile']
            dynamic_val = day_features['RollMedDollarVol'].quantile(percentile_used)
            vol_cutoff = max(vol_cutoff, dynamic_val)

        # 2. Logic Mask
        mask = (
            (day_features['RollMedDollarVol'] >= vol_cutoff) &
            (day_features['RollingStalePct'] <= thresholds['max_stale_pct']) &
            (day_features['RollingSameVolCount'] <= thresholds['max_same_vol_count'])
        )

        # 3. Capture Detailed Audit Snapshot
        if audit_container is not None:
            audit_container['date'] = valid_dates[-1]
            audit_container['total_tickers_available'] = len(day_features)
            audit_container['percentile_setting'] = percentile_used
            audit_container['percentile_value_usd'] = dynamic_val
            audit_container['final_cutoff_usd'] = vol_cutoff
            audit_container['tickers_passed'] = mask.sum()
            
            # Save the DataFrame!
            snapshot = day_features.copy()
            snapshot['Calculated_Cutoff'] = vol_cutoff
            snapshot['Passed_Vol_Check'] = snapshot['RollMedDollarVol'] >= vol_cutoff
            snapshot['Passed_Final'] = mask
            # Sort by volume so user can see the cutoff point easily
            snapshot = snapshot.sort_values('RollMedDollarVol', ascending=False)
            audit_container['universe_snapshot'] = snapshot

        return day_features[mask].index.tolist()

    def _error_result(self, msg):
        return EngineOutput(pd.Series(dtype=float), pd.Series(dtype=float), pd.DataFrame(), [], pd.Series(dtype=float), {}, pd.DataFrame(), pd.Timestamp.min, pd.Timestamp.min, pd.Timestamp.min, msg)

# ==============================================================================
# SECTION E: THE UI (Visualization)
# Update this function to read the audit data from the `debug_data` and print it nicely.
# Updated print logic to detect date shift
# Fixed EngineInput argument mapping
# ==============================================================================

def plot_walk_forward_analyzer(df_ohlcv, 
                               default_start_date='2020-01-01', 
                               default_calc_period=126, 
                               default_fwd_period=63,
                               default_metric='Sharpe (ATR)', 
                               default_rank_start=1, 
                               default_rank_end=10,
                               default_benchmark_ticker='SPY', 
                               master_calendar_ticker='SPY', 
                               quality_thresholds=None, 
                               debug=False):
    
    engine = AlphaEngine(df_ohlcv, master_ticker=master_calendar_ticker)
    results_container = [None]
    debug_container = [None]

    # --- UPDATED DEFAULT SETTINGS WITH PERCENTILE ---
    if quality_thresholds is None:
        quality_thresholds = {
            'min_median_dollar_volume': 100_000, # Hard floor
            'min_liquidity_percentile': 0.50,    # Top 50%
            'max_stale_pct': 0.05, 
            'max_same_vol_count': 10
        }

    # (Widget setup code remains the same...)
    mode_selector = widgets.RadioButtons(options=['Ranking', 'Manual List'], value='Ranking', description='Portfolio Mode:', layout={'width': 'max-content'})
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date))
    calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period:')
    fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period:')
    metric_dropdown = widgets.Dropdown(options=list(METRIC_REGISTRY.keys()), value=default_metric, description='Metric:')
    rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
    rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
    manual_tickers_input = widgets.Textarea(value='', placeholder='Enter tickers...', description='Manual Tickers:', layout={'width': '400px', 'height': '80px'})
    benchmark_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()

    ranking_controls = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input])
    manual_controls = widgets.HBox([manual_tickers_input])
    date_controls = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
    ui = widgets.VBox([mode_selector, date_controls, ranking_controls, manual_controls, widgets.HBox([benchmark_input, update_button]), ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    
    def on_mode_change(c):
        ranking_controls.layout.display = 'flex' if c['new'] == 'Ranking' else 'none'
        manual_controls.layout.display = 'none' if c['new'] == 'Ranking' else 'flex'
    mode_selector.observe(on_mode_change, names='value')
    on_mode_change({'new': mode_selector.value})

    fig = go.FigureWidget()
    fig.update_layout(title='Walk-Forward Performance Analysis', height=600, template="plotly_white", hovermode='x unified')
    for i in range(50): fig.add_trace(go.Scatter(visible=False, line=dict(width=2)))
    fig.add_trace(go.Scatter(name='Benchmark', visible=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(name='Group Portfolio', visible=True, line=dict(color='green', width=3)))

    def update_plot(b):
        ticker_list_output.clear_output()
        manual_list = [t.strip().upper() for t in manual_tickers_input.value.split(',') if t.strip()]
        start_date_raw = pd.to_datetime(start_date_picker.value)
        
        if start_date_raw < (engine.trading_calendar[0] - pd.Timedelta(days=7)):
            with ticker_list_output: print(f"⚠️ DATE WARNING: Start date {start_date_raw.date()} is too early."); return

        inputs = EngineInput(
            mode=mode_selector.value,
            start_date=start_date_raw,
            calc_period=calc_period_input.value,
            fwd_period=fwd_period_input.value,
            metric=metric_dropdown.value,
            benchmark_ticker=benchmark_input.value.strip().upper(),
            rank_start=rank_start_input.value,
            rank_end=rank_end_input.value,
            quality_thresholds=quality_thresholds,
            manual_tickers=manual_list,
            debug=debug
        )
        
        with ticker_list_output:
            res = engine.run(inputs)
            results_container[0] = res
            debug_container[0] = res.debug_data
            if res.error_msg: print(res.error_msg); return

            with fig.batch_update():
                cols = res.normalized_plot_data.columns.tolist()
                for i in range(50):
                    if i < len(cols): fig.data[i].update(x=res.normalized_plot_data.index, y=res.normalized_plot_data[cols[i]], name=cols[i], visible=True)
                    else: fig.data[i].visible = False
                
                fig.data[50].update(x=res.benchmark_series.index, y=res.benchmark_series.values, name=f"Benchmark ({inputs.benchmark_ticker})", visible=not res.benchmark_series.empty)
                fig.data[51].update(x=res.portfolio_series.index, y=res.portfolio_series.values, visible=True)
                fig.layout.shapes = [dict(type="line", x0=res.calc_end_date, y0=0, x1=res.calc_end_date, y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))]

            req_date = inputs.start_date.date()
            act_date = res.start_date.date()
            if req_date != act_date: print(f"ℹ️ Info: Start date {req_date} is not a trading day. Snapping forward to {act_date}.")
            
            # --- LIQUIDITY AUDIT PRINT ---
            if inputs.mode == 'Ranking' and res.debug_data and 'audit_liquidity' in res.debug_data:
                audit = res.debug_data['audit_liquidity']
                if audit:
                    pct_str = f"{audit.get('percentile_setting', 0)*100:.0f}%"
                    cut_val = audit.get('final_cutoff_usd', 0)
                    print("-" * 60)
                    print(f"🔍 LIQUIDITY CHECK ({act_date})")
                    print(f"   Universe Size: {audit.get('total_tickers_available')} tickers")
                    print(f"   Filtering: Top {pct_str} of Market")
                    print(f"   Calculated Cutoff: ${cut_val:,.0f} / day")
                    print(f"   Tickers Remaining: {audit.get('tickers_passed')}")
                    print("-" * 60)
            
            print(f"Analysis Period: {act_date} to {res.viz_end_date.date()}.")
            
            if inputs.mode == 'Ranking': print("Ranked Tickers:"); pprint.pprint(res.tickers)
            else: print("Manual Portfolio Tickers:"); pprint.pprint(res.tickers)
            
            m = res.perf_metrics
            rows = [
                {'Metric': 'Group Portfolio Gain', 'Full': m.get('full_p_gain'), 'Calc': m.get('calc_p_gain'), 'Fwd': m.get('fwd_p_gain')},
                {'Metric': f'Benchmark ({inputs.benchmark_ticker}) Gain', 'Full': m.get('full_b_gain'), 'Calc': m.get('calc_b_gain'), 'Fwd': m.get('fwd_b_gain')},
                {'Metric': '== Gain Delta', 'Full': m.get('full_p_gain',0)-m.get('full_b_gain',0), 'Calc': m.get('calc_p_gain',0)-m.get('calc_b_gain',0), 'Fwd': m.get('fwd_p_gain',0)-m.get('fwd_b_gain',0)},
                {'Metric': 'Group Sharpe (ATR)', 'Full': m.get('full_p_sharpe_atr'), 'Calc': m.get('calc_p_sharpe_atr'), 'Fwd': m.get('fwd_p_sharpe_atr')},
                {'Metric': f'Benchmark Sharpe (ATR)', 'Full': m.get('full_b_sharpe_atr'), 'Calc': m.get('calc_b_sharpe_atr'), 'Fwd': m.get('fwd_b_sharpe_atr')},
                {'Metric': '== Sharpe Delta', 'Full': m.get('full_p_sharpe_atr',0)-m.get('full_b_sharpe_atr',0), 'Calc': m.get('calc_p_sharpe_atr',0)-m.get('calc_b_sharpe_atr',0), 'Fwd': m.get('fwd_p_sharpe_atr',0)-m.get('fwd_b_sharpe_atr',0)}
            ]
            display(pd.DataFrame(rows).set_index('Metric').style.format("{:+.2%}", na_rep="N/A"))

    update_button.on_click(update_plot)
    update_plot(None)
    display(ui, fig)
    return results_container, debug_container

# ==============================================================================
# SECTION F: UTILITIES
# ==============================================================================

def print_nested(d, indent=0, width=4):
    """Pretty-print any nested dict/list/tuple combination."""
    spacing = ' ' * indent
    if isinstance(d, dict):
        for k, v in d.items():
            print(f'{spacing}{k}:')
            print_nested(v, indent + width, width)
    elif isinstance(d, (list, tuple)):
        for item in d:
            print_nested(item, indent, width)
    else:
        print(f'{spacing}{d}')



In [2]:
# pd.set_option('display.max_rows', None)  display all rows
pd.set_option('display.max_rows', 100)

In [3]:
import pandas as pd

data_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet'
df_ohlcv = pd.read_parquet(data_path, engine='pyarrow')
print(f'df_ohlcv.info():\n{df_ohlcv.info()}')
df_ohlcv

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9657171 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZWS', Timestamp('2025-12-03 00:00:00'))
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Adj Open   float64
 1   Adj High   float64
 2   Adj Low    float64
 3   Adj Close  float64
 4   Volume     int64  
dtypes: float64(4), int64(1)
memory usage: 405.9+ MB
df_ohlcv.info():
None


Adj Open  Adj High  Adj Low  Adj Close    Volume
Ticker Date                                                        
A      1999-11-18   27.2452   29.9398  23.9519    26.3470  74716395
       1999-11-19   25.7108   25.7482  23.8396    24.1764  18198354
       1999-11-22   24.7378   26.3470  23.9893    26.3470   7857764
       1999-11-23   25.4488   26.1225  23.9518    23.9518   7138322
       1999-11-24   24.0267   25.1120  23.9518    24.5881   5785610
...                     ...       ...      ...        ...       ...
ZWS    2025-11-26   47.5400   48.7000  47.3000    48.1300   1154100
       2025-11-28   48.4600   48.4800  47.7000    47.7000    481400
       2025-12-01   47.1700   48.1800  47.1500    47.7400    608100
       2025-12-02   47.9800   48.3100  47.7100    47.8200    838200
       2025-12-03   47.8800   48.2200  47.4200    47.4500    688255

[9657171 rows x 5 columns]

In [4]:
results_container, debug_container = plot_walk_forward_analyzer(
    df_ohlcv=df_ohlcv,
    default_start_date='2025-08-13',
    default_calc_period=10,
    default_fwd_period=5,
    default_metric='Sharpe (ATR)',
    default_rank_start=15,
    default_rank_end=16,
    default_benchmark_ticker='VOO', 
    master_calendar_ticker='VOO',    
    quality_thresholds = { 
        'min_median_dollar_volume': 100_000, # A low "hard floor" to filter absolute errors/garbage
        # If min_liquidity_percentile is 0.8 (Top 20%), we want values > the 0.8 quantile.            
        'min_liquidity_percentile': 0.50,    # Dynamic: Only keep the top 50% of stocks by volume
        'max_stale_pct': 0.05, 
        'max_same_vol_count': 10
    },
    debug=True  # <-- Activate the new mode!
)

--- ⚙️ Initializing AlphaEngine ---
Optimizing data structures...
✅ AlphaEngine Ready.


FigureWidget({
    'data': [{'line': {'width': 2},
              'name': 'JPST',
              'type': 'scatter',
              'uid': 'a03e9992-470b-4543-9bb1-859a87bdbafd',
              'visible': True,
              'x': array([datetime.datetime(2025, 8, 13, 0, 0),
                          datetime.datetime(2025, 8, 14, 0, 0),
                          datetime.datetime(2025, 8, 15, 0, 0),
                          datetime.datetime(2025, 8, 18, 0, 0),
                          datetime.datetime(2025, 8, 19, 0, 0),
                          datetime.datetime(2025, 8, 20, 0, 0),
                          datetime.datetime(2025, 8, 21, 0, 0),
                          datetime.datetime(2025, 8, 22, 0, 0),
                          datetime.datetime(2025, 8, 25, 0, 0),
                          datetime.datetime(2025, 8, 26, 0, 0),
                          datetime.datetime(2025, 8, 27, 0, 0),
                          datetime.datetime(2025, 8, 28, 0, 0),
                          

In [5]:
features_df = generate_features(df_ohlcv=df_ohlcv)

In [6]:
df_ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9657171 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZWS', Timestamp('2025-12-03 00:00:00'))
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Adj Open   float64
 1   Adj High   float64
 2   Adj Low    float64
 3   Adj Close  float64
 4   Volume     int64  
dtypes: float64(4), int64(1)
memory usage: 663.9+ MB


In [7]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9657171 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZWS', Timestamp('2025-12-03 00:00:00'))
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   TR                   float64
 1   ATR                  float64
 2   ATRP                 float64
 3   RollingStalePct      float64
 4   RollMedDollarVol     float64
 5   RollingSameVolCount  float64
dtypes: float64(6)
memory usage: 737.6+ MB


In [8]:
tickers = ['VOO', 'VLO', 'JPST']
date_start = '2025-08-13'
date_end = '2025-09-04'

In [9]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9657171 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZWS', Timestamp('2025-12-03 00:00:00'))
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   TR                   float64
 1   ATR                  float64
 2   ATRP                 float64
 3   RollingStalePct      float64
 4   RollMedDollarVol     float64
 5   RollingSameVolCount  float64
dtypes: float64(6)
memory usage: 737.6+ MB


In [ ]:
# Using the same function structure but for features_df
def get_ticker_features(
    features_df: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True
) -> Union[pd.DataFrame, dict]:
    """
    Get features data for specified tickers within a date range.
    
    Parameters
    ----------
    features_df : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and feature columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options: 
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)
    
    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered features data in specified format
    """
    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]
    
    # Filter the data using MultiIndex slicing
    try:
        filtered_data = features_df.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        if verbose:
            print(f"Error filtering data: {e}")
        return pd.DataFrame() if return_format == "dataframe" else {}
    
    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(f"No data found for tickers {tickers} in date range {date_start} to {date_end}")
        return filtered_data
    
    # Print summary if verbose
    if verbose:
        print(f"Features data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}")
        print(f"Total rows: {len(filtered_data)}")
        print(f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
              f"{filtered_data.index.get_level_values(1).max()}")
        print(f"Available features: {', '.join(filtered_data.columns.tolist())}")
        
        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")
    
    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[date_start:date_end]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result
    
    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(filtered_data.xs(ticker, level=0).loc[date_start:date_end])
            except KeyError:
                result.append(pd.DataFrame())
        return result
    
    elif return_format == "dataframe":
        return filtered_data
    
    else:
        raise ValueError(f"Invalid return_format: {return_format}. "
                         f"Must be 'dataframe', 'dict', or 'separate'")

# Define the get_ticker_OHLCV function
def get_ticker_OHLCV(
    df_ohlcv: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True
) -> Union[pd.DataFrame, dict]:
    """
    Get OHLCV data for specified tickers within a date range.
    
    Parameters
    ----------
    df_ohlcv : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and OHLCV columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options: 
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)
    
    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered OHLCV data in specified format
    
    Raises
    ------
    ValueError
        If input parameters are invalid
    KeyError
        If tickers not found in DataFrame
    
    Examples
    --------
    >>> # Get data for single ticker
    >>> vlo_data = get_ticker_OHLCV(df_ohlcv, 'VLO', '2025-08-13', '2025-09-04')
    
    >>> # Get data for multiple tickers
    >>> multi_data = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13', '2025-09-04')
    
    >>> # Get data as dictionary
    >>> data_dict = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13', 
    ...                              '2025-09-04', return_format='dict')
    """
    
    # Input validation
    if not isinstance(df_ohlcv, pd.DataFrame):
        raise TypeError("df_ohlcv must be a pandas DataFrame")
    
    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("DataFrame must have MultiIndex of (ticker, date)")
    
    if len(df_ohlcv.index.levels) != 2:
        raise ValueError("MultiIndex must have exactly 2 levels: (ticker, date)")
    
    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]
    elif not isinstance(tickers, list):
        raise TypeError("tickers must be a string or list of strings")
    
    # Convert dates to Timestamps
    try:
        start_date = pd.Timestamp(date_start)
        end_date = pd.Timestamp(date_end)
    except ValueError as e:
        raise ValueError(f"Invalid date format. Use 'YYYY-MM-DD': {e}")
    
    if start_date > end_date:
        raise ValueError("date_start must be before or equal to date_end")
    
    # Check if tickers exist in the DataFrame
    available_tickers = df_ohlcv.index.get_level_values(0).unique()
    missing_tickers = [t for t in tickers if t not in available_tickers]
    
    if missing_tickers:
        raise KeyError(f"Ticker(s) not found in DataFrame: {missing_tickers}")
    
    # Filter the data using MultiIndex slicing
    try:
        filtered_data = df_ohlcv.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        raise ValueError(f"Error filtering data: {e}")
    
    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(f"No data found for tickers {tickers} in date range {date_start} to {date_end}")
        return filtered_data
    
    # Print summary if verbose
    if verbose:
        print(f"Data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}")
        print(f"Total rows: {len(filtered_data)}")
        print(f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
              f"{filtered_data.index.get_level_values(1).max()}")
        
        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")
    
    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[date_start:date_end]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result
    
    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(filtered_data.xs(ticker, level=0).loc[date_start:date_end])
            except KeyError:
                result.append(pd.DataFrame())
        return result
    
    elif return_format == "dataframe":
        return filtered_data
    
    else:
        raise ValueError(f"Invalid return_format: {return_format}. "
                         f"Must be 'dataframe', 'dict', or 'separate'")

# Use the function to extract data
tickers = ['VOO', 'VLO', 'JPST']
tickers = ['VOO']
date_start = '2025-08-13'
date_end = '2025-09-04'



def create_combined_dict(
    df_ohlcv: pd.DataFrame,
    features_df: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    verbose: bool = True
) -> dict:
    """
    Create a combined dictionary with both OHLCV and features data for each ticker.
    
    Parameters:
    -----------
    df_ohlcv : pd.DataFrame
        DataFrame with OHLCV data (MultiIndex: ticker, date)
    features_df : pd.DataFrame
        DataFrame with features data (MultiIndex: ticker, date)
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    verbose : bool, optional
        Whether to print progress information (default: True)
    
    Returns:
    --------
    dict
        Dictionary with tickers as keys and combined DataFrames (OHLCV + features) as values
    """
    # Convert single ticker to list
    if isinstance(tickers, str):
        tickers = [tickers]
    
    if verbose:
        print(f"Creating combined dictionary for {len(tickers)} ticker(s)")
        print(f"Date range: {date_start} to {date_end}")
        print("=" * 60)
    
    # Get OHLCV data as dictionary
    ohlcv_dict = get_ticker_OHLCV(
        df_ohlcv, tickers, date_start, date_end, 
        return_format='dict', verbose=verbose
    )
    
    # Get features data as dictionary
    features_dict = get_ticker_features(
        features_df, tickers, date_start, date_end,
        return_format='dict', verbose=verbose
    )
    
    # Create combined_dict
    combined_dict = {}
    
    for ticker in tickers:
        if verbose:
            print(f"\nProcessing {ticker}...")
        
        # Check if ticker exists in both dictionaries
        if ticker in ohlcv_dict and ticker in features_dict:
            ohlcv_data = ohlcv_dict[ticker]
            features_data = features_dict[ticker]
            
            # Check if both dataframes have data
            if not ohlcv_data.empty and not features_data.empty:
                # Combine OHLCV and features data
                # Note: Both dataframes have the same index (dates), so we can concatenate
                combined_df = pd.concat([ohlcv_data, features_data], axis=1)
                
                # Ensure proper index naming
                combined_df.index.name = 'Date'
                
                # Store in combined_dict
                combined_dict[ticker] = combined_df
                
                if verbose:
                    print(f"  ✓ Successfully combined data")
                    print(f"  OHLCV shape: {ohlcv_data.shape}")
                    print(f"  Features shape: {features_data.shape}")
                    print(f"  Combined shape: {combined_df.shape}")
                    print(f"  Date range: {combined_df.index.min()} to {combined_df.index.max()}")
            else:
                if verbose:
                    print(f"  ✗ Cannot combine: One or both dataframes are empty")
                    print(f"    OHLCV empty: {ohlcv_data.empty}")
                    print(f"    Features empty: {features_data.empty}")
                combined_dict[ticker] = pd.DataFrame()
        else:
            if verbose:
                print(f"  ✗ Ticker not found in both dictionaries")
                if ticker not in ohlcv_dict:
                    print(f"    Not in OHLCV data")
                if ticker not in features_dict:
                    print(f"    Not in features data")
            combined_dict[ticker] = pd.DataFrame()
    
    # Print summary
    if verbose:
        print("\n" + "=" * 60)
        print("SUMMARY")
        print("=" * 60)
        print(f"Total tickers processed: {len(tickers)}")
        
        tickers_with_data = [ticker for ticker, df in combined_dict.items() if not df.empty]
        print(f"Tickers with combined data: {len(tickers_with_data)}")
        
        if tickers_with_data:
            print("\nTicker details:")
            for ticker in tickers_with_data:
                df = combined_dict[ticker]
                print(f"  {ticker}: {df.shape} - {df.index.min()} to {df.index.max()}")
                print(f"    Columns: {len(df.columns)}")
        
        empty_tickers = [ticker for ticker, df in combined_dict.items() if df.empty]
        if empty_tickers:
            print(f"\nTickers with no data: {', '.join(empty_tickers)}")
    
    return combined_dict

# Example usage:
if __name__ == "__main__":
    # Assuming df_ohlcv and features_df are already loaded
    
    # Define parameters
    tickers = ['VOO', 'VLO', 'JPST']
    date_start = '2025-08-13'
    date_end = '2025-09-04'
    
    # Create combined dictionary
    combined_dict = create_combined_dict(
        df_ohlcv=df_ohlcv,
        features_df=features_df,
        tickers=tickers,
        date_start=date_start,
        date_end=date_end,
        verbose=True
    )
    
    # Analyze the combined data
    print("\n" + "=" * 60)
    print("ANALYZING COMBINED DATA")
    print("=" * 60)
    
    for ticker, combined_df in combined_dict.items():
        if not combined_df.empty:
            print(f"\n{ticker}:")
            print(f"  Shape: {combined_df.shape}")
            print(f"  Date range: {combined_df.index.min()} to {combined_df.index.max()}")
            print(f"  Columns ({len(combined_df.columns)}):")
            
            # Group columns by type
            ohlcv_cols = [col for col in combined_df.columns if 'Adj' in col or col == 'Volume']
            feature_cols = [col for col in combined_df.columns if col not in ohlcv_cols]
            
            if ohlcv_cols:
                print(f"    OHLCV: {', '.join(ohlcv_cols)}")
            if feature_cols:
                print(f"    Features: {', '.join(feature_cols)}")
            
            # Check for missing values
            nan_count = combined_df.isna().sum().sum()
            if nan_count > 0:
                print(f"  Missing values: {nan_count}")
            
            # Show sample
            print(f"\n  Sample (first 3 rows):")
            print(combined_df.head(3))

In [20]:
combined_dict['VOO']

,Adj Open,Adj High,Adj Low,Adj Close,Volume,TR,ATR,ATRP,RollingStalePct,RollMedDollarVol,RollingSameVolCount
Date,,,,,,,,,,,
2025-08-13,591.169,592.336,589.125,591.149,5408423,3.241,5.207220,0.008809,0.0,2.939980e+09,0.0
2025-08-14,589.235,591.817,588.816,591.159,6254536,3.001,5.049633,0.008542,0.0,2.940885e+09,0.0
2025-08-15,592.166,592.256,588.976,589.883,7216880,3.280,4.923231,0.008346,0.0,2.946086e+09,0.0
2025-08-18,589.295,590.332,588.666,589.674,5326890,1.666,4.690571,0.007955,0.0,2.960129e+09,0.0
2025-08-19,589.564,590.431,585.266,586.453,6758573,5.165,4.724459,0.008056,0.0,2.969911e+09,0.0
2025-08-20,586.134,586.323,580.191,584.907,7133944,6.262,4.834284,0.008265,0.0,2.973949e+09,0.0
2025-08-21,583.262,584.788,580.988,582.604,4821048,3.919,4.768906,0.008186,0.0,2.973949e+09,0.0
2025-08-22,584.598,592.625,584.169,591.518,6407572,10.021,5.144056,0.008696,0.0,2.978218e+09,0.0
2025-08-25,590.362,591.538,588.866,589.016,6501540,2.672,4.967480,0.008434,0.0,2.989747e+09,0.0


In [17]:
# Using the same function structure but for features_df
def get_ticker_features(
    features_df: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True
) -> Union[pd.DataFrame, dict]:
    """
    Get features data for specified tickers within a date range.
    
    Parameters
    ----------
    features_df : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and feature columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options: 
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)
    
    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered features data in specified format
    """
    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]
    
    # Filter the data using MultiIndex slicing
    try:
        filtered_data = features_df.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        if verbose:
            print(f"Error filtering data: {e}")
        return pd.DataFrame() if return_format == "dataframe" else {}
    
    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(f"No data found for tickers {tickers} in date range {date_start} to {date_end}")
        return filtered_data
    
    # Print summary if verbose
    if verbose:
        print(f"Features data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}")
        print(f"Total rows: {len(filtered_data)}")
        print(f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
              f"{filtered_data.index.get_level_values(1).max()}")
        print(f"Available features: {', '.join(filtered_data.columns.tolist())}")
        
        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")
    
    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[date_start:date_end]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result
    
    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(filtered_data.xs(ticker, level=0).loc[date_start:date_end])
            except KeyError:
                result.append(pd.DataFrame())
        return result
    
    elif return_format == "dataframe":
        return filtered_data
    
    else:
        raise ValueError(f"Invalid return_format: {return_format}. "
                         f"Must be 'dataframe', 'dict', or 'separate'")

# Define the get_ticker_OHLCV function (assuming it exists or define it)
def get_ticker_OHLCV(
    df_ohlcv: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True
) -> Union[pd.DataFrame, dict]:
    """
    Get OHLCV data for specified tickers within a date range.
    
    Parameters
    ----------
    df_ohlcv : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and OHLCV columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options: 
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)
    
    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered OHLCV data in specified format
    
    Raises
    ------
    ValueError
        If input parameters are invalid
    KeyError
        If tickers not found in DataFrame
    
    Examples
    --------
    >>> # Get data for single ticker
    >>> vlo_data = get_ticker_OHLCV(df_ohlcv, 'VLO', '2025-08-13', '2025-09-04')
    
    >>> # Get data for multiple tickers
    >>> multi_data = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13', '2025-09-04')
    
    >>> # Get data as dictionary
    >>> data_dict = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13', 
    ...                              '2025-09-04', return_format='dict')
    """
    
    # Input validation
    if not isinstance(df_ohlcv, pd.DataFrame):
        raise TypeError("df_ohlcv must be a pandas DataFrame")
    
    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("DataFrame must have MultiIndex of (ticker, date)")
    
    if len(df_ohlcv.index.levels) != 2:
        raise ValueError("MultiIndex must have exactly 2 levels: (ticker, date)")
    
    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]
    elif not isinstance(tickers, list):
        raise TypeError("tickers must be a string or list of strings")
    
    # Convert dates to Timestamps
    try:
        start_date = pd.Timestamp(date_start)
        end_date = pd.Timestamp(date_end)
    except ValueError as e:
        raise ValueError(f"Invalid date format. Use 'YYYY-MM-DD': {e}")
    
    if start_date > end_date:
        raise ValueError("date_start must be before or equal to date_end")
    
    # Check if tickers exist in the DataFrame
    available_tickers = df_ohlcv.index.get_level_values(0).unique()
    missing_tickers = [t for t in tickers if t not in available_tickers]
    
    if missing_tickers:
        raise KeyError(f"Ticker(s) not found in DataFrame: {missing_tickers}")
    
    # Filter the data using MultiIndex slicing
    try:
        filtered_data = df_ohlcv.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        raise ValueError(f"Error filtering data: {e}")
    
    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(f"No data found for tickers {tickers} in date range {date_start} to {date_end}")
        return filtered_data
    
    # Print summary if verbose
    if verbose:
        print(f"Data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}")
        print(f"Total rows: {len(filtered_data)}")
        print(f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
              f"{filtered_data.index.get_level_values(1).max()}")
        
        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")
    
    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[date_start:date_end]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result
    
    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(filtered_data.xs(ticker, level=0).loc[date_start:date_end])
            except KeyError:
                result.append(pd.DataFrame())
        return result
    
    elif return_format == "dataframe":
        return filtered_data
    
    else:
        raise ValueError(f"Invalid return_format: {return_format}. "
                         f"Must be 'dataframe', 'dict', or 'separate'")

# Use the function to extract data
tickers = ['VOO', 'VLO', 'JPST']
tickers = ['VOO']
date_start = '2025-08-13'
date_end = '2025-09-04'

print("Extracting features data...")
print("=" * 60)

# Option 1: Get as DataFrame with MultiIndex
features_data = get_ticker_features(features_df, tickers, date_start, date_end)
print("\nFeatures data (first 10 rows):")
print(features_data.head(10))

print("\n" + "=" * 60)
print("Option 2: Get as dictionary for easier access")
print("=" * 60)

# Option 2: Get as dictionary
features_dict = get_ticker_features(features_df, tickers, date_start, date_end, 
                                    return_format='dict')

for ticker, data in features_dict.items():
    print(f"\n{ticker} features shape: {data.shape}")
    if not data.empty:
        print(f"Date range: {data.index.min()} to {data.index.max()}")
        print("First few rows:")
        print(data.head())
    else:
        print("No data available")
    print("-" * 40)

print("\n" + "=" * 60)
print("Understanding the features:")
print("=" * 60)

# Display feature descriptions
feature_descriptions = {
    'TR': 'True Range (daily volatility measure)',
    'ATR': 'Average True Range (smoothed volatility)',
    'ATRP': 'Average True Range Percentage (ATR as % of price)',
    'RollingStalePct': 'Percentage of stale prices in rolling window',
    'RollMedDollarVol': 'Rolling median dollar volume',
    'RollingSameVolCount': 'Count of consecutive days with same volume'
}

print("Feature descriptions:")
for feature, description in feature_descriptions.items():
    print(f"  {feature}: {description}")

print("\n" + "=" * 60)
print("Statistical summary of extracted features:")
print("=" * 60)

# Statistical summary
if not features_data.empty:
    print(features_data.describe())

print("\n" + "=" * 60)
print("Alternative: Direct slicing without function")
print("=" * 60)

# Direct slicing (alternative method)
direct_result = features_df.loc[(tickers, slice(date_start, date_end)), :]
print(f"Direct slicing result shape: {direct_result.shape}")
print(f"\nNumber of trading days per ticker:")
print(direct_result.index.get_level_values(0).value_counts())

# If you want to analyze specific features
print("\n" + "=" * 60)
print("Analyzing specific features:")
print("=" * 60)

if not features_data.empty:
    # Example: Look at ATR (Average True Range) across tickers
    print("\nATR (Average True Range) statistics by ticker:")
    for ticker in tickers:
        if ticker in features_data.index.get_level_values(0):
            ticker_atr = features_data.xs(ticker, level=0)['ATR']
            if not ticker_atr.empty:
                print(f"\n{ticker}:")
                print(f"  Mean ATR: {ticker_atr.mean():.4f}")
                print(f"  Max ATR: {ticker_atr.max():.4f}")
                print(f"  Min ATR: {ticker_atr.min():.4f}")
                print(f"  Std Dev: {ticker_atr.std():.4f}")

print("\n" + "=" * 60)
print("Combining with OHLCV data:")
print("=" * 60)

# Get OHLCV data for same tickers and date range
ohlcv_data = get_ticker_OHLCV(df_ohlcv, tickers, date_start, date_end, verbose=False)

# Get OHLCV data as dictionary
ohlcv_dict = get_ticker_OHLCV(df_ohlcv, tickers, date_start, date_end, 
                              return_format='dict', verbose=False)

print("\nOHLCV data per ticker:")
for ticker, data in ohlcv_dict.items():
    print(f"\n{ticker} OHLCV shape: {data.shape}")
    if not data.empty:
        print(f"Date range: {data.index.min()} to {data.index.max()}")
        print("Columns:", data.columns.tolist())
        print(data.head())

print("\n" + "=" * 60)
print("Creating combined dictionary (OHLCV + Features):")
print("=" * 60)

# Create combined_dict with both OHLCV and features for each ticker
combined_dict = {}
for ticker in tickers:
    if ticker in ohlcv_dict and ticker in features_dict:
        # Check if both dataframes have data
        if not ohlcv_dict[ticker].empty and not features_dict[ticker].empty:
            # Combine OHLCV and features data
            # Use concat to join along columns (axis=1)
            combined_df = pd.concat([ohlcv_dict[ticker], features_dict[ticker]], axis=1)
            
            # Ensure the index is proper (dates)
            combined_df.index.name = 'Date'
            
            # Add to combined_dict
            combined_dict[ticker] = combined_df
            
            print(f"\n{ticker} - Combined data:")
            print(f"  Shape: {combined_df.shape}")
            print(f"  Date range: {combined_df.index.min()} to {combined_df.index.max()}")
            print(f"  Total columns: {len(combined_df.columns)}")
            print(f"  Columns: {combined_df.columns.tolist()}")
            print(f"\n  Sample data (first 5 rows):")
            print(combined_df.head())
        else:
            print(f"\n{ticker} - Cannot combine: One or both dataframes are empty")
            print(f"  OHLCV empty: {ohlcv_dict[ticker].empty}")
            print(f"  Features empty: {features_dict[ticker].empty}")
            combined_dict[ticker] = pd.DataFrame()
    else:
        print(f"\n{ticker} - Not found in both dictionaries")
        combined_dict[ticker] = pd.DataFrame()

print("\n" + "=" * 60)
print("Analyzing combined data in combined_dict:")
print("=" * 60)

# Analyze the combined data
for ticker, combined_df in combined_dict.items():
    if not combined_df.empty:
        print(f"\n{ticker} combined data analysis:")
        print(f"  Total rows: {len(combined_df)}")
        print(f"  Columns: {combined_df.columns.tolist()}")
        
        # Check for any NaN values
        nan_counts = combined_df.isna().sum()
        if nan_counts.sum() > 0:
            print(f"  NaN values by column:")
            for col, count in nan_counts.items():
                if count > 0:
                    print(f"    {col}: {count} NaN values ({count/len(combined_df)*100:.2f}%)")
        else:
            print(f"  No NaN values found")
        
        # Basic statistics for key columns
        print(f"\n  Key statistics:")
        if 'Adj Close' in combined_df.columns:
            print(f"    Adj Close - Mean: ${combined_df['Adj Close'].mean():.2f}")
            print(f"    Adj Close - Std: ${combined_df['Adj Close'].std():.2f}")
        if 'Volume' in combined_df.columns:
            print(f"    Volume - Mean: {combined_df['Volume'].mean():,.0f}")
        if 'ATR' in combined_df.columns:
            print(f"    ATR - Mean: {combined_df['ATR'].mean():.4f}")
        if 'ATRP' in combined_df.columns:
            print(f"    ATRP - Mean: {combined_df['ATRP'].mean():.4f}%")
        
        print(f"\n  First few rows of combined data:")
        print(combined_df.head(3))
    else:
        print(f"\n{ticker}: No combined data available")

print("\n" + "=" * 60)
print("Additional functionality: Accessing specific data from combined_dict")
print("=" * 60)

# Example: Access specific ticker's combined data
if 'VOO' in combined_dict and not combined_dict['VOO'].empty:
    print("\nAccessing VOO's combined data:")
    voo_data = combined_dict['VOO']
    print(f"Shape: {voo_data.shape}")
    
    # Calculate daily returns
    if 'Adj Close' in voo_data.columns:
        voo_data['Daily_Return'] = voo_data['Adj Close'].pct_change()
        print(f"\nVOO Daily Returns Statistics:")
        print(f"  Mean Daily Return: {voo_data['Daily_Return'].mean()*100:.4f}%")
        print(f"  Std Daily Return: {voo_data['Daily_Return'].std()*100:.4f}%")
        print(f"  Max Daily Return: {voo_data['Daily_Return'].max()*100:.4f}%")
        print(f"  Min Daily Return: {voo_data['Daily_Return'].min()*100:.4f}%")
    
    # Check correlation between volume and ATR
    if 'Volume' in voo_data.columns and 'ATR' in voo_data.columns:
        correlation = voo_data['Volume'].corr(voo_data['ATR'])
        print(f"\nCorrelation between Volume and ATR: {correlation:.4f}")

print("\n" + "=" * 60)
print("Exporting combined_dict data:")
print("=" * 60)

# Option to export each ticker's combined data to CSV
export_to_csv = False  # Set to True if you want to export
if export_to_csv:
    for ticker, combined_df in combined_dict.items():
        if not combined_df.empty:
            filename = f"{ticker}_combined_{date_start}_{date_end}.csv"
            combined_df.to_csv(filename)
            print(f"Exported {ticker} data to {filename}")

print("\nSummary of combined_dict:")
print(f"Number of tickers in combined_dict: {len(combined_dict)}")
tickers_with_data = [ticker for ticker, df in combined_dict.items() if not df.empty]
print(f"Tickers with combined data: {tickers_with_data}")
if tickers_with_data:
    for ticker in tickers_with_data:
        print(f"  {ticker}: {combined_dict[ticker].shape}")

Extracting features data...
Features data retrieved for 1 ticker(s) from 2025-08-13 to 2025-09-04
Total rows: 16
Date range in data: 2025-08-13 00:00:00 to 2025-09-04 00:00:00
Available features: TR, ATR, ATRP, RollingStalePct, RollMedDollarVol, RollingSameVolCount
  VOO: 16 rows

Features data (first 10 rows):
                       TR       ATR      ATRP  RollingStalePct  \
Ticker Date                                                      
VOO    2025-08-13   3.241  5.207220  0.008809              0.0   
       2025-08-14   3.001  5.049633  0.008542              0.0   
       2025-08-15   3.280  4.923231  0.008346              0.0   
       2025-08-18   1.666  4.690571  0.007955              0.0   
       2025-08-19   5.165  4.724459  0.008056              0.0   
       2025-08-20   6.262  4.834284  0.008265              0.0   
       2025-08-21   3.919  4.768906  0.008186              0.0   
       2025-08-22  10.021  5.144056  0.008696              0.0   
       2025-08-25   2.672  

In [ ]:
combined_dict

In [ ]:
# Using the same function structure but for features_df
def get_ticker_features(
    features_df: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True
) -> Union[pd.DataFrame, dict]:
    """
    Get features data for specified tickers within a date range.
    
    Parameters
    ----------
    features_df : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and feature columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options: 
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)
    
    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered features data in specified format
    """
    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]
    
    # Filter the data using MultiIndex slicing
    try:
        filtered_data = features_df.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        if verbose:
            print(f"Error filtering data: {e}")
        return pd.DataFrame() if return_format == "dataframe" else {}
    
    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(f"No data found for tickers {tickers} in date range {date_start} to {date_end}")
        return filtered_data
    
    # Print summary if verbose
    if verbose:
        print(f"Features data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}")
        print(f"Total rows: {len(filtered_data)}")
        print(f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
              f"{filtered_data.index.get_level_values(1).max()}")
        print(f"Available features: {', '.join(filtered_data.columns.tolist())}")
        
        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")
    
    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[date_start:date_end]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result
    
    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(filtered_data.xs(ticker, level=0).loc[date_start:date_end])
            except KeyError:
                result.append(pd.DataFrame())
        return result
    
    elif return_format == "dataframe":
        return filtered_data
    
    else:
        raise ValueError(f"Invalid return_format: {return_format}. "
                         f"Must be 'dataframe', 'dict', or 'separate'")

# Use the function to extract data
tickers = ['VOO', 'VLO', 'JPST']
date_start = '2025-08-13'
date_end = '2025-09-04'

print("Extracting features data...")
print("=" * 60)

# Option 1: Get as DataFrame with MultiIndex
features_data = get_ticker_features(features_df, tickers, date_start, date_end)
print("\nFeatures data (first 10 rows):")
print(features_data.head(10))

print("\n" + "=" * 60)
print("Option 2: Get as dictionary for easier access")
print("=" * 60)

# Option 2: Get as dictionary
features_dict = get_ticker_features(features_df, tickers, date_start, date_end, 
                                    return_format='dict')

for ticker, data in features_dict.items():
    print(f"\n{ticker} features shape: {data.shape}")
    if not data.empty:
        print(f"Date range: {data.index.min()} to {data.index.max()}")
        print("First few rows:")
        print(data.head())
    else:
        print("No data available")
    print("-" * 40)

print("\n" + "=" * 60)
print("Understanding the features:")
print("=" * 60)

# Display feature descriptions
feature_descriptions = {
    'TR': 'True Range (daily volatility measure)',
    'ATR': 'Average True Range (smoothed volatility)',
    'ATRP': 'Average True Range Percentage (ATR as % of price)',
    'RollingStalePct': 'Percentage of stale prices in rolling window',
    'RollMedDollarVol': 'Rolling median dollar volume',
    'RollingSameVolCount': 'Count of consecutive days with same volume'
}

print("Feature descriptions:")
for feature, description in feature_descriptions.items():
    print(f"  {feature}: {description}")

print("\n" + "=" * 60)
print("Statistical summary of extracted features:")
print("=" * 60)

# Statistical summary
if not features_data.empty:
    print(features_data.describe())

print("\n" + "=" * 60)
print("Alternative: Direct slicing without function")
print("=" * 60)

# Direct slicing (alternative method)
direct_result = features_df.loc[(tickers, slice(date_start, date_end)), :]
print(f"Direct slicing result shape: {direct_result.shape}")
print(f"\nNumber of trading days per ticker:")
print(direct_result.index.get_level_values(0).value_counts())

# If you want to analyze specific features
print("\n" + "=" * 60)
print("Analyzing specific features:")
print("=" * 60)

if not features_data.empty:
    # Example: Look at ATR (Average True Range) across tickers
    print("\nATR (Average True Range) statistics by ticker:")
    for ticker in tickers:
        if ticker in features_data.index.get_level_values(0):
            ticker_atr = features_data.xs(ticker, level=0)['ATR']
            if not ticker_atr.empty:
                print(f"\n{ticker}:")
                print(f"  Mean ATR: {ticker_atr.mean():.4f}")
                print(f"  Max ATR: {ticker_atr.max():.4f}")
                print(f"  Min ATR: {ticker_atr.min():.4f}")
                print(f"  Std Dev: {ticker_atr.std():.4f}")

print("\n" + "=" * 60)
print("Combining with OHLCV data:")
print("=" * 60)

# If you also have OHLCV data, you can combine them
# (Assuming you have the get_ticker_OHLCV function from earlier)

try:
    # Get OHLCV data for same tickers and date range
    ohlcv_data = get_ticker_OHLCV(df_ohlcv, tickers, date_start, date_end, verbose=False)
    
    # Combine features with OHLCV data
    if not ohlcv_data.empty and not features_data.empty:
        # Since both have the same MultiIndex, we can combine them
        combined_data = pd.concat([ohlcv_data, features_data], axis=1)
        print(f"\nCombined data shape: {combined_data.shape}")
        print(f"Total columns: {len(combined_data.columns)}")
        print(f"Columns: {combined_data.columns.tolist()}")
        
        # Show combined data for a specific ticker
        if 'VOO' in combined_data.index.get_level_values(0):
            print(f"\nSample of combined data for VOO:")
            print(combined_data.xs('VOO', level=0).head())
except NameError:
    print("Note: OHLCV data not available in current context")

In [ ]:
features_dict

In [ ]:
combined_data

In [ ]:
# Simple one-liner to extract the data
features_subset = features_df.loc[(['VOO', 'VLO', 'JPST'], slice('2025-08-13', '2025-09-04')), :]
print(features_subset)

# Save to CSV
features_subset.to_csv('extracted_features.csv')

# # Save to Excel
# features_subset.to_excel('extracted_features.xlsx')

# # Save to pickle (preserves MultiIndex)
# features_subset.to_pickle('extracted_features.pkl')

In [ ]:
import pandas as pd
from typing import Union, List

def get_ticker_OHLCV(
    df_ohlcv: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True
) -> Union[pd.DataFrame, dict]:
    """
    Get OHLCV data for specified tickers within a date range.
    
    Parameters
    ----------
    df_ohlcv : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and OHLCV columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options: 
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)
    
    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered OHLCV data in specified format
    
    Raises
    ------
    ValueError
        If input parameters are invalid
    KeyError
        If tickers not found in DataFrame
    
    Examples
    --------
    >>> # Get data for single ticker
    >>> vlo_data = get_ticker_OHLCV(df_ohlcv, 'VLO', '2025-08-13', '2025-09-04')
    
    >>> # Get data for multiple tickers
    >>> multi_data = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13', '2025-09-04')
    
    >>> # Get data as dictionary
    >>> data_dict = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13', 
    ...                              '2025-09-04', return_format='dict')
    """
    
    # Input validation
    if not isinstance(df_ohlcv, pd.DataFrame):
        raise TypeError("df_ohlcv must be a pandas DataFrame")
    
    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("DataFrame must have MultiIndex of (ticker, date)")
    
    if len(df_ohlcv.index.levels) != 2:
        raise ValueError("MultiIndex must have exactly 2 levels: (ticker, date)")
    
    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]
    elif not isinstance(tickers, list):
        raise TypeError("tickers must be a string or list of strings")
    
    # Convert dates to Timestamps
    try:
        start_date = pd.Timestamp(date_start)
        end_date = pd.Timestamp(date_end)
    except ValueError as e:
        raise ValueError(f"Invalid date format. Use 'YYYY-MM-DD': {e}")
    
    if start_date > end_date:
        raise ValueError("date_start must be before or equal to date_end")
    
    # Check if tickers exist in the DataFrame
    available_tickers = df_ohlcv.index.get_level_values(0).unique()
    missing_tickers = [t for t in tickers if t not in available_tickers]
    
    if missing_tickers:
        raise KeyError(f"Ticker(s) not found in DataFrame: {missing_tickers}")
    
    # Filter the data using MultiIndex slicing
    try:
        filtered_data = df_ohlcv.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        raise ValueError(f"Error filtering data: {e}")
    
    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(f"No data found for tickers {tickers} in date range {date_start} to {date_end}")
        return filtered_data
    
    # Print summary if verbose
    if verbose:
        print(f"Data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}")
        print(f"Total rows: {len(filtered_data)}")
        print(f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
              f"{filtered_data.index.get_level_values(1).max()}")
        
        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")
    
    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[date_start:date_end]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result
    
    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(filtered_data.xs(ticker, level=0).loc[date_start:date_end])
            except KeyError:
                result.append(pd.DataFrame())
        return result
    
    elif return_format == "dataframe":
        return filtered_data
    
    else:
        raise ValueError(f"Invalid return_format: {return_format}. "
                         f"Must be 'dataframe', 'dict', or 'separate'")


# Example usage with your data
if __name__ == "__main__":
    # Your example data
    tickers = ['VLO', 'JPST']
    date_start = '2025-08-13'
    date_end = '2025-09-04'
    
    print("Example 1: Default usage (returns DataFrame)")
    print("=" * 60)
    result1 = get_ticker_OHLCV(df_ohlcv, tickers, date_start, date_end)
    print("\nFirst 5 rows:")
    print(result1.head())
    
    print("\n" + "=" * 60)
    print("Example 2: Return as dictionary")
    print("=" * 60)
    result2 = get_ticker_OHLCV(df_ohlcv, tickers, date_start, date_end, 
                               return_format='dict', verbose=True)
    
    for ticker, data in result2.items():
        print(f"\n{ticker} data shape: {data.shape}")
        print(data.head() if not data.empty else "Empty DataFrame")
    
    print("\n" + "=" * 60)
    print("Example 3: Single ticker")
    print("=" * 60)
    vlo_data = get_ticker_OHLCV(df_ohlcv, 'VLO', date_start, date_end, verbose=True)
    
    print("\n" + "=" * 60)
    print("Example 4: With error handling")
    print("=" * 60)
    try:
        # This should raise an error if 'INVALID' is not in your DataFrame
        invalid_data = get_ticker_OHLCV(df_ohlcv, ['VLO', 'INVALID'], date_start, date_end)
    except KeyError as e:
        print(f"Error caught: {e}")
    
    print("\n" + "=" * 60)
    print("Example 5: Silent mode (verbose=False)")
    print("=" * 60)
    silent_result = get_ticker_OHLCV(df_ohlcv, tickers, date_start, date_end, verbose=False)
    print(f"Returned DataFrame shape: {silent_result.shape}")

In [ ]:
import pandas as pd

# Create the date range filter
date_filter = (df_ohlcv.index.get_level_values(1) >= pd.Timestamp(date_start)) & \
              (df_ohlcv.index.get_level_values(1) <= pd.Timestamp(date_end))

# Filter by tickers AND date range
filtered_df = df_ohlcv.loc[(tickers, slice(date_start, date_end)), :]

# Alternative method using boolean indexing
# filtered_df = df_ohlcv[df_ohlcv.index.get_level_values(0).isin(tickers) & date_filter]

# Print the filtered data
print(f"Data for tickers {tickers} from {date_start} to {date_end}:")
print("-" * 50)

for ticker in tickers:
    print(f"\nTicker: {ticker}")
    print("-" * 30)
    
    # Extract data for current ticker
    ticker_data = filtered_df.xs(ticker, level=0)
    
    if not ticker_data.empty:
        print(ticker_data)
        print(f"\nNumber of rows for {ticker}: {len(ticker_data)}")
    else:
        print(f"No data found for {ticker} in the specified date range")
    
    print()

# If you want a summary
print("=" * 50)
print("SUMMARY:")
print(f"Total rows in filtered data: {len(filtered_df)}")
print(f"Date range: {date_start} to {date_end}")

In [ ]:
print_nested(results_container)
print('='*20)
print_nested(debug_container)

In [ ]:
debug_container

In [ ]:
my_tickers = ['SPY', 'AAPL', 'IWM', 'QQQ', 'META', 'EEM', 'BABA']
my_tickers = ['NTES', 'LII',]


In [ ]:
for ticker in my_tickers:
  if ticker in features_df.index.get_level_values('Ticker'):
    ticker_features = features_df.loc[ticker]
    ticker_features.to_csv(f'./export_csv/features_{ticker}.csv')
    print(f"✅ {ticker} features exported to: ./export_csv/features_{ticker}.csv")
  else:
    print(f"⚠️ {ticker} not found in features_df")

In [ ]:
for ticker in my_tickers:
  if ticker in df_ohlcv.index.get_level_values('Ticker'):
    ticker_features = df_ohlcv.loc[ticker]
    ticker_features.to_csv(f'./export_csv/ohlcv_{ticker}.csv')
    print(f"✅ {ticker} OHLCV exported to: ./export_csv/ohlcv_{ticker}.csv")
  else:
    print(f"⚠️ {ticker} not found in df_ohlcv")

In [ ]:
# Assuming you have run the variables setup from the previous step
snapshot_df = debug_container[0]['audit_liquidity']['universe_snapshot']

if 'AAPL' in snapshot_df.index:
    display(snapshot_df.loc[['AAPL']])
else:
    print("AAPL was not present in the data for this date.")

In [ ]:
snapshot_df.to_csv('./export_csv/snapshot_df.csv')
print(f"✅ Snapshot exported to: ./export_csv/snapshot_df.csv")
print(f"   Shape: {snapshot_df.shape}")
print(f"   Columns: {list(snapshot_df.columns)}")

In [ ]:
# 1. Access the data inside the container list
current_debug_data = debug_container[0]

# 2. Check if the audit data exists (it is created only in 'Ranking' mode)
if current_debug_data and 'audit_liquidity' in current_debug_data:
    audit = current_debug_data['audit_liquidity']
    snapshot_df = audit['universe_snapshot']
    
    print(f"📅 Date: {audit['date'].date()}")
    print(f"💰 Calculated Cutoff: ${audit['final_cutoff_usd']:,.0f}")
    print("-" * 30)

# 3. View the tickers right around the cutoff point
# Find the index where 'Passed_Vol_Check' switches from True to False
    try:
        # Get the integer location (iloc) of the last True value
        last_pass_iloc = np.where(snapshot_df['Passed_Vol_Check'])[0][-1]
        
        # Show 5 rows before and 5 rows after the cutoff
        start = max(0, last_pass_iloc - 5)
        end = min(len(snapshot_df), last_pass_iloc + 6)
        
        display(snapshot_df.iloc[start:end].style.format({
            'RollMedDollarVol': '${:,.0f}',
            'Calculated_Cutoff': '${:,.0f}',
            'RollingStalePct': '{:.1%}'
        }))
    except IndexError:
        print("Could not determine cutoff boundary (maybe all passed or all failed).")
        display(snapshot_df.head())
else:
    print("⚠️ No audit data found. Make sure you are in 'Ranking' mode and have clicked 'Update Chart'.")    

In [ ]:
display(snapshot_df)

In [ ]:
================================  
================================  
================================  